In [2]:
from torch.utils.data import Dataset
import torch
import pandas as pd
import json
import os
from pathlib import Path
import torch
import torch.nn as nn

In [3]:
class LSTMTCNRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, lstm_layers=1, tcn_channels=[64, 32]):
        super(LSTMTCNRegressor, self).__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=lstm_layers, batch_first=True)
        
        # TCN part: using 1D Convolutions
        self.tcn = nn.Sequential(
            nn.Conv1d(hidden_dim, tcn_channels[0], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(tcn_channels[0], tcn_channels[1], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # output shape: (batch, channels, 1)
        )
        
        self.fc = nn.Linear(tcn_channels[-1], 1)
    
    def forward(self, x):
        # x shape: (batch, time_steps, features)
        out, _ = self.lstm(x)  # (batch, time_steps, hidden_dim)
        out = out.permute(0, 2, 1)  # (batch, hidden_dim, time_steps) for Conv1D
        out = self.tcn(out)  # (batch, channels, 1)
        out = out.squeeze(-1)  # (batch, channels)
        out = self.fc(out)  # (batch, 1)
        return out.squeeze(-1)

Loading the model